In [76]:
# _importing required libraries
import os
import collections

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
%matplotlib inline

In [77]:
# _initializing constant variables
input_file_path = os.getcwd() + f'/../../data/processed_data.csv'
subject101_file_path = os.getcwd() + f'/../../data/output_csv/subject101_processed.csv'
col_names = ['timestamp (s)', 'activityID', 'X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2']

In [78]:
# _loading preprocessed data to main dataframe
main_df = pd.read_csv(input_file_path, header=None, names=col_names)
main_df

,timestamp (s),activityID,X1,Y1,Z1,X2,Y2,Z2
0,37.66,1,2.21530,8.27915,5.58753,-0.004750,0.037579,-0.011145
1,37.67,1,2.29196,7.67288,5.74467,-0.171710,0.025479,-0.009538
2,37.68,1,2.29090,7.14240,5.82342,-0.238241,0.011214,0.000831
3,37.69,1,2.21800,7.14365,5.89930,-0.192912,0.019053,0.013374
4,37.70,1,2.30106,7.25857,6.09259,-0.069961,-0.018328,0.004582
...,...,...,...,...,...,...,...,...
288214,974.50,3,-1.99794,3.94300,9.15686,-0.112651,-0.003501,-0.066523
288215,974.51,3,-2.00276,4.20689,8.96346,-0.179092,0.020300,-0.010637
288216,974.52,3,-1.96042,4.09413,9.11842,-0.285215,0.010714,-0.017346
288217,974.53,3,-1.97860,3.78659,8.65734,-0.347098,0.040476,-0.020536


In [79]:
subject101_df = pd.read_csv(subject101_file_path, names=col_names, skiprows=1)
subject101_df

,timestamp (s),activityID,X1,Y1,Z1,X2,Y2,Z2
0,37.66,1,2.21530,8.27915,5.58753,-0.004750,0.037579,-0.011145
1,37.67,1,2.29196,7.67288,5.74467,-0.171710,0.025479,-0.009538
2,37.68,1,2.29090,7.14240,5.82342,-0.238241,0.011214,0.000831
3,37.69,1,2.21800,7.14365,5.89930,-0.192912,0.019053,0.013374
4,37.70,1,2.30106,7.25857,6.09259,-0.069961,-0.018328,0.004582
...,...,...,...,...,...,...,...,...
72331,761.45,3,2.35899,7.40504,5.55414,-0.292110,0.095687,-0.024721
72332,761.46,3,2.36942,7.18043,5.90144,-0.330916,0.107007,-0.044081
72333,761.47,3,2.17406,6.83943,5.78420,-0.130746,0.074366,-0.051712
72334,761.48,3,2.17325,7.67241,5.58894,0.050723,0.035534,0.000376


# Generating subsequences for each sequence of the data

In [80]:
def elbow_techique(sub_sequence_data):
    
    # _only for manual testing
    distortions = []
    for k in range(1,10):
        kmeanModel = KMeans(n_clusters=k)
        kmeanModel.fit(sub_sequence_data)
        distortions.append(kmeanModel.inertia_)

    # _plot the distortions to observe the elbow point from the graph
    plt.figure(figsize=(16,8))
    plt.plot(K, distortions, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Distortion')
    plt.title('The Elbow Method showing the optimal k')
    plt.show()

In [81]:
def get_cluster_ids(sub_sequence_data, cluster_cnt):
    
    # _passing random state as an argument to make Kmeans clustering deterministic
    kMeans_model = KMeans(n_clusters = cluster_cnt,random_state=1234).fit(sub_sequence_data)
    return kMeans_model

In [82]:
def get_assigned_words(seq_clusters, cluster_words):
    
    # _assign word to each cluster of the subsequence usnig numpy where function
    assigned_words = np.where(seq_clusters != 0, seq_clusters, cluster_words[0])
    for idx in range(1, len(cluster_words)):
         assigned_words = np.where(seq_clusters != idx, assigned_words, cluster_words[idx])
            
    return assigned_words

In [83]:
def generating_numbers(sequence_names):
    
    # _each word contains common prefix of subsequence and unique cluster alphabet
    alphabet = 'ABCDEFGHIJ'
    words_dict = {}
    
    for seq in sequence_names:
        prefix = seq
        words_dict[seq] = [prefix+'_'+alphabet[i] for i in range(len(alphabet))]
        
    return words_dict

In [84]:
# _initializing variables
window_length = 50
window_overlap = int(window_length/2)

sequence_names = col_names[1:]
num_of_subsequences = len(sequence_names)
sub_sequences = [[] for x in range(num_of_subsequences)]
sub_sequences_101 = [[] for x in range(num_of_subsequences)]

In [85]:
window_index = 0
max_window_index = len(main_df.index)

while window_index <= (max_window_index - window_length):

    activity_sequence = main_df[sequence_names[0]][window_index:window_index+window_length].tolist()

    if len(set(activity_sequence)) == 1:
        sub_sequences[0].append(activity_sequence[0])
        
        for idx in range(1, num_of_subsequences):
            sub_sequences[idx].append(main_df[sequence_names[idx]][window_index:window_index+window_length].tolist())
            
    window_index += window_overlap

# _converting into numpy arrays
np_sequences = np.asarray(sub_sequences[1:])
print(np_sequences.shape)

(6, 11506, 50)


In [86]:
window_index = 0
max_window_index = len(subject101_df.index)

while window_index <= (max_window_index - window_length):

    activity_sequence = subject101_df[sequence_names[0]][window_index:window_index+window_length].tolist()

    if len(set(activity_sequence)) == 1:
        sub_sequences_101[0].append(activity_sequence[0])
        
        for idx in range(1, num_of_subsequences):
            sub_sequences_101[idx].append(subject101_df[sequence_names[idx]][window_index:window_index+window_length].tolist())
            
    window_index += window_overlap

# _converting into numpy arrays
np_sequences_101 = np.asarray(sub_sequences_101[1:])
print(np_sequences_101.shape)

(6, 2888, 50)


# Finding the statistics of the clusters(mean,variance,skewness)


In [87]:
# _document dataframe to store word assignments of each window
doc_df = pd.DataFrame(columns=col_names[1:])
doc_df['activityID'] = sub_sequences_101[0]
doc_df

,activityID,X1,Y1,Z1,X2,Y2,Z2
0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NaN,NaN,NaN,NaN,NaN,NaN
3,1,NaN,NaN,NaN,NaN,NaN,NaN
4,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2883,3,NaN,NaN,NaN,NaN,NaN,NaN
2884,3,NaN,NaN,NaN,NaN,NaN,NaN
2885,3,NaN,NaN,NaN,NaN,NaN,NaN
2886,3,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
words_dict = generating_numbers(sequence_names[1:])
print(words_dict)
print()

# _after observing the clustering result with elbow method, we are manually choosing the n_clusters value.
# _testing: elbow_techique(np_sequences[0])
sequence_cluster_cnts = {'X1': 3, 'Y1': 4, 'Z1': 3, 'X2': 3, 'Y2': 3, 'Z2': 3}
print(sequence_cluster_cnts)

{'X1': ['X1_A', 'X1_B', 'X1_C', 'X1_D', 'X1_E', 'X1_F', 'X1_G', 'X1_H', 'X1_I', 'X1_J'], 'Y1': ['Y1_A', 'Y1_B', 'Y1_C', 'Y1_D', 'Y1_E', 'Y1_F', 'Y1_G', 'Y1_H', 'Y1_I', 'Y1_J'], 'Z1': ['Z1_A', 'Z1_B', 'Z1_C', 'Z1_D', 'Z1_E', 'Z1_F', 'Z1_G', 'Z1_H', 'Z1_I', 'Z1_J'], 'X2': ['X2_A', 'X2_B', 'X2_C', 'X2_D', 'X2_E', 'X2_F', 'X2_G', 'X2_H', 'X2_I', 'X2_J'], 'Y2': ['Y2_A', 'Y2_B', 'Y2_C', 'Y2_D', 'Y2_E', 'Y2_F', 'Y2_G', 'Y2_H', 'Y2_I', 'Y2_J'], 'Z2': ['Z2_A', 'Z2_B', 'Z2_C', 'Z2_D', 'Z2_E', 'Z2_F', 'Z2_G', 'Z2_H', 'Z2_I', 'Z2_J']}

{'X1': 3, 'Y1': 4, 'Z1': 3, 'X2': 3, 'Y2': 3, 'Z2': 3}


In [89]:
for idx, (seq, cluster_cnt) in enumerate(sequence_cluster_cnts.items()):
    
    # _perform k means clustering on subsequences
    KMeans_models = get_cluster_ids(np_sequences[idx], cluster_cnt)
    seq_clusters = KMeans_models.predict(np_sequences_101[idx])
    
    cluster_words = words_dict[seq][:cluster_cnt]
    
    print(f'{idx} -- {seq} -- {cluster_cnt} -- {cluster_words} -- {set(seq_clusters)}')
    
    # _get assigned words and fill the values in doc_df
    assigned_words = get_assigned_words(seq_clusters, cluster_words)
    doc_df[seq] = assigned_words

0 -- X1 -- 3 -- ['X1_A', 'X1_B', 'X1_C'] -- {0, 1, 2}
1 -- Y1 -- 4 -- ['Y1_A', 'Y1_B', 'Y1_C', 'Y1_D'] -- {0, 1, 2, 3}
2 -- Z1 -- 3 -- ['Z1_A', 'Z1_B', 'Z1_C'] -- {0, 1, 2}
3 -- X2 -- 3 -- ['X2_A', 'X2_B', 'X2_C'] -- {0, 1, 2}
4 -- Y2 -- 3 -- ['Y2_A', 'Y2_B', 'Y2_C'] -- {0, 1, 2}
5 -- Z2 -- 3 -- ['Z2_A', 'Z2_B', 'Z2_C'] -- {0, 1, 2}


doc_df

In [90]:
doc_df

,activityID,X1,Y1,Z1,X2,Y2,Z2
0,1,X1_C,Y1_B,Z1_B,X2_B,Y2_A,Z2_A
1,1,X1_C,Y1_B,Z1_B,X2_B,Y2_A,Z2_A
2,1,X1_C,Y1_D,Z1_B,X2_B,Y2_A,Z2_A
3,1,X1_C,Y1_D,Z1_B,X2_B,Y2_B,Z2_A
4,1,X1_C,Y1_B,Z1_B,X2_B,Y2_B,Z2_A
...,...,...,...,...,...,...,...
2883,3,X1_C,Y1_B,Z1_A,X2_A,Y2_A,Z2_A
2884,3,X1_C,Y1_B,Z1_B,X2_B,Y2_A,Z2_A
2885,3,X1_C,Y1_B,Z1_B,X2_B,Y2_A,Z2_A
2886,3,X1_C,Y1_B,Z1_B,X2_B,Y2_A,Z2_A


In [91]:
# doc_df.drop_duplicates(keep=False, inplace=True)
# doc_df

# Finding the statistics of the clusters(mean,variance,skewness)

In [92]:
window_cols = ['Col_'+str(idx) for idx in range(1, window_length+2)]

In [93]:
def clust_subseq(X,i):
    d1=pd.DataFrame(doc_df[X])
    X1_df = pd.DataFrame(np_sequences_101[i])
    df=pd.concat([d1,X1_df],axis = 1)
    df.columns = window_cols.copy()
    return df

In [94]:
cluster_subseq =[]
di = {'X1':0,'Y1':1,'Z1':2,'X2':3,'Y2':4,'Z2':5}

for key,value in di.items():    
    last = clust_subseq(key,value)
    cluster_subseq.append(last)
cluster_subseq

[     Col_1    Col_2    Col_3    Col_4    Col_5    Col_6    Col_7    Col_8  \
 0     X1_C  2.21530  2.29196  2.29090  2.21800  2.30106  2.07165  2.41148   
 1     X1_C  2.26407  2.30000  2.21613  2.21372  2.06309  2.25765  2.29891   
 2     X1_C  2.48446  2.41664  2.56915  2.29091  2.27665  2.61702  2.54759   
 3     X1_C  2.05921  1.77559  1.79642  2.18428  2.48855  2.65852  2.73626   
 4     X1_C  1.36652  1.61009  1.62105  1.81320  1.81106  1.58619  1.32032   
 ...    ...      ...      ...      ...      ...      ...      ...      ...   
 2883  X1_C  1.96764  1.97753  1.90731  1.86177  1.93870  1.86099  2.01081   
 2884  X1_C  2.66816  2.74667  2.58963  2.82358  2.57750  2.65842  2.66723   
 2885  X1_C  2.49063  2.52414  2.48956  2.64258  2.64632  2.57021  2.54525   
 2886  X1_C  2.25660  2.37450  2.37207  2.55781  2.43803  2.32520  2.35659   
 2887  X1_C  2.48438  2.32762  2.35579  2.27914  2.39037  2.39491  2.43938   
 
          Col_9    Col_10  ...    Col_42    Col_43    Col_44  

In [95]:
def clusterstatistics(Y,X):
    
    cluster_df = Y.loc[(Y['Col_1'] == X)] 
    
    Columns = window_cols[1:]
    Mean = []
    Variance = []
    Skewness = []

    for values in Columns:
        M = cluster_df[values].mean()
        V = cluster_df[values].var()
        S = cluster_df[values].skew()
        Mean.append(M)
        Variance.append(V)
        Skewness.append(S)
    
    stat_df=pd.DataFrame(Mean,columns=["Mean"])
    stat_df["Variance"]=Variance
    stat_df["Skewness"]=Skewness
    return  stat_df

In [96]:
statistic=[]
dic = {0:['X1_A', 'X1_B', 'X1_C'],
       1:['Y1_A', 'Y1_B', 'Y1_C', 'Y1_D'],
       2:['Z1_A', 'Z1_B', 'Z1_C'],
       3:['X2_A', 'X2_B', 'X2_C'],
       4:['Y2_A', 'Y2_B', 'Y2_C'],
       5:['Z2_A', 'Z2_B', 'Z2_C']
      }
for i in range(len(cluster_subseq)):
    cluster_names=dic[i]
    for j in range(len(cluster_names)):
        last1 = clusterstatistics(cluster_subseq[i],cluster_names[j])
        statistic.append(last1)
print(statistic)

[        Mean  Variance  Skewness
0   5.655582  2.961667 -0.672421
1   5.655851  2.960424 -0.640884
2   5.661213  2.913346 -0.549435
3   5.668651  2.900966 -0.509940
4   5.671863  2.872975 -0.500146
5   5.677325  2.862474 -0.465550
6   5.683580  2.815874 -0.430875
7   5.689706  2.814755 -0.412895
8   5.702729  2.808278 -0.403485
9   5.701433  2.796300 -0.402872
10  5.701702  2.792676 -0.390900
11  5.695405  2.796867 -0.385491
12  5.695214  2.820809 -0.413037
13  5.692894  2.820956 -0.422612
14  5.690554  2.800308 -0.418041
15  5.687318  2.776696 -0.435505
16  5.687230  2.748087 -0.433108
17  5.680783  2.780286 -0.444332
18  5.683936  2.810081 -0.468948
19  5.686957  2.800106 -0.461740
20  5.680486  2.804976 -0.480989
21  5.685980  2.821116 -0.497301
22  5.681281  2.809714 -0.515910
23  5.679916  2.811867 -0.553935
24  5.677397  2.804178 -0.587459
25  5.679256  2.819995 -0.622367
26  5.677015  2.835119 -0.597593
27  5.677990  2.814685 -0.510785
28  5.682807  2.815179 -0.471997
29  5.686

In [97]:
allvalues=[]
for i in range(len(statistic)):
    l=statistic[i].T
    allvalues.append(l)
    
allvalues1 = []
for i in range(len(allvalues)):
    aa = pd.DataFrame(allvalues[i].stack().to_frame().values).T
    allvalues1.append(aa)

In [98]:
embeddings_filepath = os.getcwd() + f'/../../data/sub_sequence_output/word_embeddings_from_clusters.txt'
pd.concat(allvalues1).to_csv(embeddings_filepath, index=False, header= False)

In [99]:
# _combine individual words as documents
doc_df['final_sub_sequence'] = doc_df[col_names[2:]].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
doc_df

,activityID,X1,Y1,Z1,X2,Y2,Z2,final_sub_sequence
0,1,X1_C,Y1_B,Z1_B,X2_B,Y2_A,Z2_A,X1_C Y1_B Z1_B X2_B Y2_A Z2_A
1,1,X1_C,Y1_B,Z1_B,X2_B,Y2_A,Z2_A,X1_C Y1_B Z1_B X2_B Y2_A Z2_A
2,1,X1_C,Y1_D,Z1_B,X2_B,Y2_A,Z2_A,X1_C Y1_D Z1_B X2_B Y2_A Z2_A
3,1,X1_C,Y1_D,Z1_B,X2_B,Y2_B,Z2_A,X1_C Y1_D Z1_B X2_B Y2_B Z2_A
4,1,X1_C,Y1_B,Z1_B,X2_B,Y2_B,Z2_A,X1_C Y1_B Z1_B X2_B Y2_B Z2_A
...,...,...,...,...,...,...,...,...
2883,3,X1_C,Y1_B,Z1_A,X2_A,Y2_A,Z2_A,X1_C Y1_B Z1_A X2_A Y2_A Z2_A
2884,3,X1_C,Y1_B,Z1_B,X2_B,Y2_A,Z2_A,X1_C Y1_B Z1_B X2_B Y2_A Z2_A
2885,3,X1_C,Y1_B,Z1_B,X2_B,Y2_A,Z2_A,X1_C Y1_B Z1_B X2_B Y2_A Z2_A
2886,3,X1_C,Y1_B,Z1_B,X2_B,Y2_A,Z2_A,X1_C Y1_B Z1_B X2_B Y2_A Z2_A


In [100]:
# _save the combined values to text files
for activity in doc_df['activityID'].unique():
    output_filepath = os.getcwd() + f'/../../data/sub_sequence_output/activity_subseq_' + str(activity) + '.txt'
    doc_df.loc[doc_df['activityID'] == activity][['final_sub_sequence']].to_csv(output_filepath, sep='\t', index=False, header= False)